In [11]:
# Uploading libraries
import ast
import pandas as pd
import seaborn as sns
import numpy as np
from datasets import load_dataset
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from adjustText import adjust_text


# Uploading dataset
df_2020_q1 = pd.read_csv('Original_data_Q1_2019_Q1_2020/Divvy_Trips_2020_Q1.csv')


In [12]:
df_2020_q1.sample(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
208838,467A31C38585B0A2,docked_bike,2020-02-06 09:00:32,2020-02-06 09:04:52,Clinton St & Lake St,66,Franklin St & Illinois St,672.0,41.8856,-87.6418,41.8910,-87.6355,member
380762,5429E34640F2C5D1,docked_bike,2020-03-06 18:25:12,2020-03-06 18:31:05,Bissell St & Armitage Ave,113,Marshfield Ave & Cortland St,58.0,41.9184,-87.6522,41.9160,-87.6689,member
415067,E5D990AD06CD96D2,docked_bike,2020-03-28 16:28:21,2020-03-28 16:58:02,Damen Ave & Charleston St,310,Rush St & Superior St,161.0,41.9201,-87.6779,41.8958,-87.6259,member
188634,BCE0637195BABF20,docked_bike,2020-02-01 07:36:30,2020-02-01 07:55:42,Franklin St & Lake St,164,Clark St & Drummond Pl,220.0,41.8858,-87.6355,41.9312,-87.6443,member
200378,FA441EBC094277B4,docked_bike,2020-02-06 17:05:14,2020-02-06 17:20:11,Orleans St & Merchandise Mart Plaza,100,Dayton St & North Ave,60.0,41.8882,-87.6364,41.9106,-87.6494,member


In [13]:
# Dropped unused calumns

columns_to_drop = ['rideable_type', 'start_station_name',	'start_station_id',	'end_station_name',	'end_station_id',	'start_lat',	'start_lng',	'end_lat',	'end_lng']

df_clean_2020_q1 = df_2020_q1.drop(columns=columns_to_drop)

df_clean_2020_q1.sample(5)

,ride_id,started_at,ended_at,member_casual
376003,80AD3F88FAE3F125,2020-03-12 18:26:50,2020-03-12 18:35:12,member
299629,C8773953E9152C73,2020-03-05 08:16:15,2020-03-05 08:21:13,member
413876,5DECCE63802A4664,2020-03-18 10:40:33,2020-03-18 11:04:04,member
396818,BF22D563FCE4DF02,2020-03-31 16:46:37,2020-03-31 16:47:13,member
424489,C76CC7985B097CD1,2020-03-01 17:13:43,2020-03-01 17:17:21,member


In [14]:
# trip_id	start_time	end_time	usertype
# Renamed columns to make mergeable with other datasets in analysis project

column_mapping = {'ride_id': 'trip_id',
                  'started_at': 'start_time',
                  'ended_at': 'end_time',
                  'member_casual': 'usertype'
                  }
df_clean_2020_q1.rename(columns=column_mapping, inplace=True)





In [17]:
df_clean_2020_q1.sample(10)

,trip_id,start_time,end_time,usertype
183000,3D891548141961B0,2020-02-06 19:52:53,2020-02-06 20:07:30,member
14807,A50AEEA6D8C0E9FC,2020-01-20 17:25:48,2020-01-20 17:38:59,member
7934,1D54B9222FE957E7,2020-01-14 07:08:05,2020-01-14 07:09:59,member
60901,CEAE5DBDD4B64284,2020-01-06 14:13:32,2020-01-06 14:18:19,member
171509,91997DBC80C91A02,2020-02-19 13:08:26,2020-02-19 13:12:50,member
175287,B934D6622AC12CD0,2020-02-25 11:51:54,2020-02-25 12:00:20,member
245080,3918BDBAE9392382,2020-02-01 12:02:33,2020-02-01 12:13:28,member
210786,26E9863E79B3DD1F,2020-02-26 08:39:27,2020-02-26 08:44:31,member
382499,307B714B58AAC02B,2020-03-21 14:51:08,2020-03-21 15:11:53,member
413161,D1CD178F796B05A6,2020-03-05 16:58:39,2020-03-05 17:04:12,member


In [18]:
# Formatted columns start_time & end_time from Series to date&time then created new column where calculated trip duration

df_clean_2020_q1['start_time'] = pd.to_datetime(df_clean_2020_q1['start_time'], format='%Y-%m-%d %H:%M:%S')
df_clean_2020_q1['end_time'] = pd.to_datetime(df_clean_2020_q1['end_time'], format='%Y-%m-%d %H:%M:%S')


df_clean_2020_q1['ride_length_seconds'] = (df_clean_2020_q1['end_time'] - df_clean_2020_q1['start_time']).dt.total_seconds()

df_clean_2020_q1['ride_length_seconds'] = df_clean_2020_q1['ride_length_seconds'].astype(int)


def seconds_to_hhmmss(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


df_clean_2020_q1['ride_length'] = df_clean_2020_q1['ride_length_seconds'].apply(seconds_to_hhmmss)


df_clean_2020_q1.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length
414038,5BC6B4098CB934C1,2020-03-25 14:14:34,2020-03-25 14:27:32,member,778,00:12:58
271483,74AEB79A03B3D553,2020-02-17 08:19:27,2020-02-17 08:27:42,member,495,00:08:15
136932,23294242F9C4FFEB,2020-01-09 08:30:35,2020-01-09 09:16:25,member,2750,00:45:50
79854,069CC3444628B80C,2020-01-31 06:26:46,2020-01-31 06:43:09,member,983,00:16:23
195913,A4395E606201B602,2020-02-26 18:20:29,2020-02-26 18:23:23,member,174,00:02:54
180322,7518281B65710163,2020-02-03 11:52:11,2020-02-03 12:00:28,member,497,00:08:17
422162,0F1BD33E7B29017F,2020-03-08 17:27:53,2020-03-08 17:47:14,member,1161,00:19:21
161278,6D70E0FDAD8E35C5,2020-02-26 17:09:27,2020-02-26 17:35:11,member,1544,00:25:44
32034,193A5A109417AB41,2020-01-17 18:29:33,2020-01-17 18:35:52,member,379,00:06:19
297769,50F1AFC87E7D22E7,2020-03-08 21:09:55,2020-03-08 21:30:50,member,1255,00:20:55


In [19]:
df_clean_2020_q1['day_of_week'] = df_clean_2020_q1['start_time'].dt.day_name()


df_clean_2020_q1.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length,day_of_week
221595,02A3FBCF9E2A8889,2020-02-25 09:30:18,2020-02-25 09:45:23,member,905,00:15:05,Tuesday
17754,1CAF865E4F78DCEE,2020-01-06 14:24:46,2020-01-06 14:27:54,member,188,00:03:08,Monday
405823,6CDD3347BFBA4243,2020-03-10 07:57:47,2020-03-10 08:03:46,member,359,00:05:59,Tuesday
346722,7861C9AFADB22917,2020-03-04 09:59:15,2020-03-04 10:12:10,member,775,00:12:55,Wednesday
97862,AF1031D49CA1BBC2,2020-01-15 17:33:08,2020-01-15 17:42:24,member,556,00:09:16,Wednesday
64898,6F62A29720B18C44,2020-01-09 06:53:30,2020-01-09 07:08:42,member,912,00:15:12,Thursday
243989,5F39C815008DC52C,2020-02-04 08:32:42,2020-02-04 08:37:19,member,277,00:04:37,Tuesday
129418,A1709B489995101B,2020-01-14 06:40:15,2020-01-14 06:54:38,member,863,00:14:23,Tuesday
61032,334370E166456511,2020-01-21 08:34:09,2020-01-21 08:40:08,member,359,00:05:59,Tuesday
45294,1E45F959C88EE012,2020-01-27 18:26:38,2020-01-27 18:31:02,member,264,00:04:24,Monday


In [20]:
df_clean_2020_q1.to_csv('Cleaned_2020_Q1.csv', index=False)